In [1]:
from pyspark.sql.functions import col,udf,monotonically_increasing_id, row_number
from pyspark.sql import SQLContext, types
from pyspark.sql.functions import *
from pyspark.sql import functions as F, SparkSession
from pyspark.sql import Window
import pyspark.sql.functions as psf
from functools import reduce
from pyspark.sql import DataFrame
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from pyspark.sql.types import *

In [2]:
input_path='/ge/gamelogs/codm_426/ingame/2021-06-28/codmvng_midasbuy_portal_cst.*'
ds = '2021-06-28'

In [3]:
#yesterday = datetime.strptime(ds, '%Y-%m-%d').date() - timedelta(days=1)
#yesterday = str(yesterday)[:10]
df_now = spark.read.csv(input_path, sep='|', mode="DROPMALFORMED", header=False)
#input_path_yesterday = input_path.replace(ds, yesterday)
#df_yesterday = spark.read.csv(input_path_yesterday, sep='|', mode="DROPMALFORMED", header=False)
df = df_now#.union(df_yesterday)

df = df.withColumnRenamed("_c0", "logdate") \
    .withColumnRenamed("_c1", "fsequence_no") \
    .withColumnRenamed("_c2", "ftran_time") \
    .withColumnRenamed("_c3", "ftran_type") \
    .withColumnRenamed("_c4", "fpay_type") \
    .withColumnRenamed("_c5", "fpay_chan") \
    .withColumnRenamed("_c6", "fpay_sub_chan") \
    .withColumnRenamed("_c9", "forder_no") \
    .withColumnRenamed("_c10", "fstatus") \
    .withColumnRenamed("_c15", "user_num") \
    .withColumnRenamed("_c16", "ftran_amt") \
    .withColumnRenamed("_c17", "ftran_info") \
    .withColumnRenamed("_c18", "ftran_detail") \
    .withColumnRenamed("_c19", "ftran_bak") \
    .withColumnRenamed("_c26", "ext_str1") \
    .withColumnRenamed("_c28", "isource") \
    .withColumnRenamed("_c29", "ftran_real_amt")

df = df.withColumn('os', F.when((F.col("fpay_chan")) == "iap", "ios")
                   .when((F.col("fpay_chan")) == "gwallet", "android")
                   .otherwise('web'))

df = df.withColumn("ftran_time", F.from_unixtime(F.col('ftran_time'))).withColumn('time',F.substring('ftran_time',1,10))
df = df.filter(df.time == ds).drop('time')
df = df.withColumn("ds", F.lit(ds))

In [4]:
df.show(1)

+--------+--------------------+-------------------+----------+---------+---------+-------------+---------+----+---------+-------+----+----+----------+----------+----------+---------+----------+------------+--------------------+----+-------------+------------+----------+----+----+--------------------+--------------------+-------+--------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-------+----------------+----+---------+--------------------+----+-------+---+----------+
| logdate|        fsequence_no|         ftran_time|ftran_type|fpay_type|fpay_chan|fpay_sub_chan|      _c7| _c8|forder_no|fstatus|_c11|_c12|      _c13|      _c14|  user_num|ftran_amt|ftran_info|ftran_detail|           ftran_bak|_c20|         _c21|        _c22|      _c23|_c24|_c25|            ext_str1|                _c27|isource|ftran_real_amt|_c30|_c31|_c32|_c33|_c34|_c35|_c36|_c37|_c38|_c39|_c40|_c41|_c42|_c43|_c44|   _c45|            _c46|_c47|     _c48|                _c49|_c50|   _

In [5]:
df.filter(df.ftran_time.contains('2021-06-28 16:')).show()

+-------+------------+----------+----------+---------+---------+-------------+---+---+---------+-------+----+----+----+----+--------+---------+----------+------------+---------+----+----+----+----+----+----+--------+----+-------+--------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+---+---+
|logdate|fsequence_no|ftran_time|ftran_type|fpay_type|fpay_chan|fpay_sub_chan|_c7|_c8|forder_no|fstatus|_c11|_c12|_c13|_c14|user_num|ftran_amt|ftran_info|ftran_detail|ftran_bak|_c20|_c21|_c22|_c23|_c24|_c25|ext_str1|_c27|isource|ftran_real_amt|_c30|_c31|_c32|_c33|_c34|_c35|_c36|_c37|_c38|_c39|_c40|_c41|_c42|_c43|_c44|_c45|_c46|_c47|_c48|_c49|_c50|_c51| os| ds|
+-------+------------+----------+----------+---------+---------+-------------+---+---+---------+-------+----+----+----+----+--------+---------+----------+------------+---------+----+----+----+----+----+----+--------+----+-------+--------------+----+----+----+----+----+----+

In [15]:
df.groupBy(df.ds).sum('ftran_amt').show(truncate=False)

+----------+--------------+
|ds        |sum(ftran_amt)|
+----------+--------------+
|2021-06-28|2112630826    |
+----------+--------------+



In [78]:
df.count()

508

In [79]:
df.show(1,truncate=False)

+--------+-------------------------------+-------------------+----------+---------+---------+-------------+---------+----+---------+-------+----+----+----------+----------+----------+---------+----------+------------+----------------------------------------------------------------+----+--------------+-----------+-----------+----+----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [80]:
df.select('ftran_time').orderBy(df.ftran_time.desc()).show()

+-------------------+
|         ftran_time|
+-------------------+
|2021-07-28 06:58:03|
|2021-07-28 06:57:33|
|2021-07-28 06:35:16|
|2021-07-28 06:29:15|
|2021-07-28 05:30:11|
|2021-07-28 04:25:17|
|2021-07-28 04:22:16|
|2021-07-28 03:46:05|
|2021-07-28 03:13:28|
|2021-07-28 03:13:11|
|2021-07-28 02:54:08|
|2021-07-28 02:53:51|
|2021-07-28 02:41:37|
|2021-07-28 02:39:58|
|2021-07-28 02:36:00|
|2021-07-28 02:23:31|
|2021-07-28 02:15:59|
|2021-07-28 02:15:14|
|2021-07-28 02:14:01|
|2021-07-28 02:13:32|
+-------------------+
only showing top 20 rows



In [30]:
iap_item_id_pattern = r'IapProductId=(\S+?)&'
item_id_pattern = r'productid=(\S+?)&'
transaction_id_pattern_iap = r'ProvideReturn=.*?(GPA\.\d+-\d+-\d+-\d+)'
provide_openid_pattern = r'provide_openid=(\d+?)&'
transaction_id_pattern_webpay = r'UserProvideMsg=.+?(\d+?)&'

android_currency_type_pattern = r'CurrencyType=(\S+?)&'
android_amt_pattern = r'CurrencyAmt=([-+]?\d*\.\d+|\d+)&'

ios_currency_type_pattern = r'IapCurrencyType=(\S+?)&'
ios_amt_pattern = r'IapAmt=([-+]?\d*\.\d+|\d+)&'

df_result = df.select('*', F.regexp_extract('ext_str1', iap_item_id_pattern, 1).alias('iap_product_id'),
                      F.regexp_extract('ext_str1', item_id_pattern, 1).alias('product_id'),
                      F.regexp_extract('ext_str1', provide_openid_pattern, 1).alias('provide_openid'),
                      F.when((F.col("fpay_chan")) == "iap",
                             F.regexp_extract('ftran_bak', ios_currency_type_pattern, 1))
                                .otherwise(F.regexp_extract('ftran_bak', android_currency_type_pattern, 1))
                                    .alias('bak_currency_type'),
                      F.when((F.col("fpay_chan")) == "iap",
                             F.regexp_extract('ftran_bak', ios_amt_pattern, 1))
                                .otherwise(F.regexp_extract('ftran_bak', android_amt_pattern, 1))
                                    .alias('bak_amt'),
                      F.regexp_extract('ext_str1', transaction_id_pattern_iap, 1).alias('transanctionidiap'),
                      F.regexp_extract('ext_str1', transaction_id_pattern_webpay, 1).alias('transanctionidwebpay')
                      )

In [31]:
df_result_selected = df_result.select('fsequence_no', 'ftran_time', 'ftran_type',
                                      'fpay_type', 'fpay_chan', 'fpay_sub_chan',
                                      'forder_no', 'fstatus', 'user_num', 'os',
                                      'ftran_amt', 'ftran_info', 'ftran_detail',
                                      'isource', 'ftran_real_amt',
                                      'iap_product_id', 'product_id', 'provide_openid',
                                      'transanctionidiap', 'transanctionidwebpay',
                                      'bak_currency_type', 'bak_amt',
                                      'ds')
df_result_selected=df_result_selected.withColumn('bak_amt', df_result_selected['bak_amt'].cast(IntegerType()))

In [32]:
df_result_selected.show(1,truncate=False)

+-------------------------------+-------------------+----------+---------+---------+-------------+---------+-------+----------+---+---------+----------+------------+-------+--------------+-------------------+-------------------+--------------------+-----------------+--------------------+-----------------+-------+----------+
|fsequence_no                   |ftran_time         |ftran_type|fpay_type|fpay_chan|fpay_sub_chan|forder_no|fstatus|user_num  |os |ftran_amt|ftran_info|ftran_detail|isource|ftran_real_amt|iap_product_id     |product_id         |provide_openid      |transanctionidiap|transanctionidwebpay|bak_currency_type|bak_amt|ds        |
+-------------------------------+-------------------+----------+---------+---------+-------------+---------+-------+----------+---+---------+----------+------------+-------+--------------+-------------------+-------------------+--------------------+-----------------+--------------------+-----------------+-------+----------+
|-APP97801-20210628-9H

In [33]:
df_result_selected.groupBy(df_result_selected.ds).sum('bak_amt').show(truncate=False)

+----------+------------+
|ds        |sum(bak_amt)|
+----------+------------+
|2021-06-28|9520000     |
+----------+------------+



In [69]:
df_result_selected.count()

440

In [70]:
df_result_selected.distinct().count()

440